In [108]:
import pandas as pd
from matplotlib import pyplot as plt

In [109]:
def modify_date(original_date, part_to_mod='sec', amount='0', func=lambda a,b:a):
    i = 0 if part_to_mod=='hour' else 1 if part_to_mod=='min' else 2
    
    splits = original_date.split('T')
    time_parts = splits[1].split(':')
    time_parts[i] = str(func(int(time_parts[i]), amount))
    
    return splits[0] + 'T' + time_parts[0] + ':' + time_parts[1] + ':' + time_parts[2]
    
def add_hours(original_date, n=0):
    return modify_date(original_date, 'hour', n, lambda a,b:a + b)

def add_minutes(original_date, n=0):   
    return modify_date(original_date, 'min', n, lambda a,b:a + b)

def sub_minutes(original_date, n=0):   
    return modify_date(original_date, 'min', n, lambda a,b:a - b)

def add_seconds(original_date, n=0):   
    return modify_date(original_date, 'sec', n, lambda a,b:a + b)

In [126]:
gpu_info = pd.read_csv('experiments.csv', index_col='id')
gpu_info['gpu-utilization'] = gpu_info['gpu-utilization'].astype(object)
gpu_info['gpu-memory'] = gpu_info['gpu-memory'].astype(object)
gpu_info['gpu-power-usage'] = gpu_info['gpu-power-usage'].astype(object)
gpu_info['gpu-temperature'] = gpu_info['gpu-temperature'].astype(object)

In [111]:
grafana_gpu_power_usage = pd.read_csv('grafana_gpu_power_usage.csv', sep=';')

In [112]:
grafana_gpu_utilization = pd.read_csv('grafana_gpu_utilization.csv', sep=';')

In [113]:
grafana_gpu_memory = pd.read_csv('grafana_gpu_memory.csv', sep=';')

In [114]:
grafana_gpu_temperature = pd.read_csv('grafana_gpu_temperature.csv', sep=';')

In [115]:
import pytz
import datetime
from datetime import timezone
from numpy import trapz
import numpy as np

def get_grafana_info(gpu_info, grafana_df, tolerance_min=15):
    grafana_df['Time'] = pd.to_datetime(grafana_df['Time'], format='%Y-%m-%dT%H:%M:%S')
    grafana_df['Time'] = grafana_df['Time'].dt.tz_convert(tz='Etc/GMT-2')

    start_time = pd.date_range(gpu_info['start-time'], periods=1) + pd.Timedelta(minutes=-tolerance_min)
    end_time = pd.date_range(gpu_info['end-time'], periods=1) + pd.Timedelta(minutes=tolerance_min)

    new_start = pd.date_range(start_time[0].replace(tzinfo=timezone.utc), periods=1)
    new_end = pd.date_range(end_time[0].replace(tzinfo=timezone.utc), periods=1)
    
    res_df = grafana_df.loc[(grafana_df['Series'] == gpu_info['nvidia-plugins'] + '-gpu0') & \
                            (grafana_df['Time'] > new_start[0]) & \
                            (grafana_df['Time'] <= new_end[0])]
    
    values = res_df['Value'].values
    print(len(values))
    
    #plt.plot(range(0, len(values)), values)
    #plt.show()
    
    area = trapz(values, range(0, len(values)), 1)
    print("\tarea =", area)
    
    return values

In [128]:
def print_grafana_info(values, label='info'):
    if all(v == 0 for v in values):
        print('\tERROR')
    else:
        print('\t' + label + ' = ' + str(max(values)))

for index, row in gpu_info.iterrows():
    print(row['network'], row['batch-size'], row['nvidia-plugins'])
    tolerance_min = 15
    
    gpu_utilization = get_grafana_info(row, grafana_gpu_utilization, tolerance_min)
    print_grafana_info(gpu_utilization, 'gpu_utilization')
    gpu_info.at[index, 'gpu-utilization'] = gpu_utilization
    
    gpu_memory = get_grafana_info(row, grafana_gpu_memory)
    print_grafana_info(gpu_memory, 'gpu_memory')
    gpu_info.at[index, 'gpu-memory'] = gpu_memory
        
    gpu_power_usage = get_grafana_info(row, grafana_gpu_power_usage, tolerance_min)
    print_grafana_info(gpu_power_usage, 'gpu_power_usage')
    gpu_info.at[index, 'gpu-power-usage'] = gpu_power_usage
    
    gpu_temperature = get_grafana_info(row, grafana_gpu_temperature, tolerance_min)
    print_grafana_info(gpu_temperature, 'gpu_temperature')
    gpu_info.at[index, 'gpu-temperature'] = gpu_temperature
        
    print()

gpu_info.to_csv('experiments-with-grafana.csv')

resnet 32 nvidia-gpu-device-plugin-j8nl9
31
	area = 60.0
	gpu_utilization = 60.0
31
	area = 479.497008658741
	gpu_memory = 95.37514802909828
62
	area = 1801312.5
	gpu_power_usage = 53985.0
62
	area = 2188.5
	gpu_temperature = 43.0

alexnet 32 nvidia-gpu-device-plugin-sn4v2
33
	area = 6.0
	gpu_utilization = 6.0
33
	area = 670.2473047169376
	gpu_memory = 95.37514802909828
65
	area = 1908229.0
	gpu_power_usage = 41608.0
65
	area = 2069.0
	gpu_temperature = 34.0

inception 32 nvidia-gpu-device-plugin-kcqcb
31
	area = 98.0
	gpu_utilization = 98.0
31
	area = 479.50316051737127
	gpu_memory = 95.37514802909828
61
	area = 2010408.5
	gpu_power_usage = 255608.0
61
	area = 1990.0
	gpu_temperature = 49.0

resnet 64 nvidia-gpu-device-plugin-j8nl9
31
	area = 0.0
	ERROR
31
	area = 478.15936389781757
	gpu_memory = 95.35054059457714
61
	area = 1850294.5
	gpu_power_usage = 169985.0
61
	area = 2156.5
	gpu_temperature = 48.0

alexnet 64 nvidia-gpu-device-plugin-bdtt6
33
	area = 0.0
	ERROR
33
	area = 670.07